<a href="https://colab.research.google.com/github/Dark-Sied/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent Recognition Dolores V01
Using a new vocabulary and training set.  
Intent Recognition is based on Intent_classification_final
Created by Christoph Windheuser, April 2020

In [1]:
import numpy as np
import pandas as pd
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
import os
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [26]:
# DEFINE GLOBAL VARIABLES:
NUM_SENT = 0
NUM_INTENTS = 0
NUM_INTENTS_UNIQUE = 0
VOCABULARY_SIZE = 0
MAX_SENT_LENGTH = 0

In [27]:
# SHOW GLOBAL VARIABLES
print ("NUM_SENT:           ", NUM_SENT)
print ("NUM_INTENTS:        ", NUM_INTENTS)
print ("NUM_INTENTS_UNIQUE: ", NUM_INTENTS_UNIQUE)
print ("VOCABULARY_SIZE:    ", VOCABULARY_SIZE)
print ("MAX_SENT_LENGTH:    ", MAX_SENT_LENGTH)

NUM_SENT:            0
NUM_INTENTS:         0
NUM_INTENTS_UNIQUE:  0
VOCABULARY_SIZE:     0
MAX_SENT_LENGTH:     0


# Read the data

In [28]:
# df = pd.read_csv("Dolores_Dataset_v01.csv", encoding = "latin1", names = ["Sentence", "Intent"])
df = pd.read_csv("mowgli_train_new.csv", encoding = "latin1", names = ["Sentence", "Intent"])
print(df.head())
intents            = df["Intent"]
NUM_INTENTS        = len(list(df["Intent"]))
intents_unique     = list(set(df["Intent"]))
NUM_INTENTS_UNIQUE = len(intents_unique)
sentences          = list(df["Sentence"])
NUM_SENT           = len(sentences)


                    Sentence           Intent
0         are you a machine?  are_you_a_robot
1  how are the things going?      how_are_you
2             nah not for me             deny
3           What's going on?      how_are_you
4             are you a bot?  are_you_a_robot


In [29]:
print ("Unique Intents: ")
print (intents_unique)
print ("Num of unique Intents: ", len(intents_unique))


Unique Intents: 
['skills', 'how_are_you', 'leave_budget', 'greet', 'insult', 'sorry', 'personal_question', 'goodbye', 'confirm', 'are_you_a_robot', 'what_is_your_name', 'conversation_restart', 'deny', 'thanks']
Num of unique Intents:  14


In [30]:
df.head()

,Sentence,Intent
0,are you a machine?,are_you_a_robot
1,how are the things going?,how_are_you
2,nah not for me,deny
3,What's going on?,how_are_you
4,are you a bot?,are_you_a_robot


In [31]:
df.shape

(346, 2)

In [32]:
print(sentences[:5])

['are you a machine?', 'how are the things going?', 'nah not for me', "What's going on?", 'are you a bot?']


# Word Cleaning
re.sub is a routine from the "Regular Expression" Library.     
r'string' means that this is a "raw string", where backslashes are treated as charachters.    
re.sub(r'[^ a-z A-Z 0-9]', " ", s) means that all characters exept a-z, A-Z and 0-9 will be replaced by space.

In [33]:
clean_sent = []
for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    clean_sent.append([i.lower() for i in w])

print (clean_sent[:5])
print("Len of clean_sent: ", len(clean_sent))


[['are', 'you', 'a', 'machine'], ['how', 'are', 'the', 'things', 'going'], ['nah', 'not', 'for', 'me'], ['what', 's', 'going', 'on'], ['are', 'you', 'a', 'bot']]
Len of clean_sent:  346


### Documentation for Tokenizer:
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [34]:
token = Tokenizer(filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
token.fit_on_texts(clean_sent)
word_index = token.word_index
VOCABULARY_SIZE = len(word_index) + 1
MAX_SENT_LENGTH = len(max(clean_sent, key = len))

#print("Vocab Size = %d. Maximum sent length = %d" % (vocab_size, max_sent_length))
print ("Max sent length: ", MAX_SENT_LENGTH)
encoded_sent = token.texts_to_sequences(clean_sent)
padded_sent = pad_sequences(encoded_sent, maxlen = MAX_SENT_LENGTH, padding = "post")


Max sent length:  12


In [35]:
# print (encoded_sent)
# print ("Num of Sentences: ", len(encoded_sent))

In [36]:
padded_sent[:5]

array([[  4,   1,  23, 129,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,   4,  60,  61,  24,   0,   0,   0,   0,   0,   0,   0],
       [ 89,  31,  21,  15,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7,  10,  24,  73,   0,   0,   0,   0,   0,   0,   0,   0],
       [  4,   1,  23,  46,   0,   0,   0,   0,   0,   0,   0,   0]],
      dtype=int32)

In [37]:
print("Shape of padded sent = ",padded_sent.shape)

Shape of padded sent =  (346, 12)


# Tokenizing the intents

In [38]:
#tokenizer for the intents
token_intents = Tokenizer(filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
token_intents.fit_on_texts(intents_unique)


In [39]:
token_intents.word_index

{'skills': 1,
 'how_are_you': 2,
 'leave_budget': 3,
 'greet': 4,
 'insult': 5,
 'sorry': 6,
 'personal_question': 7,
 'goodbye': 8,
 'confirm': 9,
 'are_you_a_robot': 10,
 'what_is_your_name': 11,
 'conversation_restart': 12,
 'deny': 13,
 'thanks': 14}

In [40]:
encoded_output = token_intents.texts_to_sequences(intents)


In [41]:
# print (encoded_output)


In [42]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [43]:
# print (encoded_output)
type (encoded_output)

numpy.ndarray

In [44]:
encoded_output.shape

(346, 1)

In [45]:
one_hot = OneHotEncoder(sparse = False)
output_one_hot = one_hot.fit_transform(encoded_output)

In [46]:
output_one_hot.shape

(346, 14)

# Define Training- and Testset

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
# train_X, val_X, train_Y, val_Y = train_test_split(padded_sent, output_one_hot, shuffle = True, test_size = 0.1)
train_X = padded_sent
train_Y = output_one_hot
val_X   = train_X
val_Y   = train_Y


In [49]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (346, 12) and train_Y = (346, 14)
Shape of val_X = (346, 12) and val_Y = (346, 14)


# Define Embeddings (from glove)

In [50]:
GLOVE_DIR  = "/Users/cwindheu/gensim-data/glove-wiki-gigaword-200/"
GLOVE_FILE = "glove-wiki-gigaword-200.txt"
EMBEDDING_DIM = 200

embeddings_index = {}

f = open(os.path.join(GLOVE_DIR, GLOVE_FILE))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400001 word vectors.


In [51]:
embedding_matrix = np.random.uniform(-1, 1, (VOCABULARY_SIZE, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be initialized randomly.
        embedding_matrix[i] = embedding_vector
    else:
        print ("Not in vacabulary: ", word)
                    

Not in vacabulary:  chatbot
Not in vacabulary:  amayzing
Not in vacabulary:  heeey
Not in vacabulary:  helloooo
Not in vacabulary:  jojojo
Not in vacabulary:  thanx
Not in vacabulary:  hellllooooooo
Not in vacabulary:  hellooo
Not in vacabulary:  hiihihi
Not in vacabulary:  thnx
Not in vacabulary:  heyho
Not in vacabulary:  hiii
Not in vacabulary:  sweatheart
Not in vacabulary:  heyo
Not in vacabulary:  ayyyy
Not in vacabulary:  whaddup


# Defining the Model

In [52]:
model = Sequential()
model.add(Embedding(VOCABULARY_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], input_length = MAX_SENT_LENGTH, trainable = True))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(NUM_INTENTS_UNIQUE, activation = "softmax"))

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 200)           56800     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               336896    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                462       
Total params: 402,382
Trainable params: 402,382
Non-trainable par

# Training the Model

In [60]:
import time

filename = 'dir_01.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

start = time.time()

hist = model.fit(train_X, train_Y, epochs = 500, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

print("Elapsed time in seconds: ", time.time() - start)


Train on 346 samples, validate on 346 samples
Epoch 1/500
346/346 [==============================] - 2s 5ms/step - loss: 0.0675 - acc: 0.9740 - val_loss: 0.0042 - val_acc: 0.9971

Epoch 00001: val_loss improved from inf to 0.00415, saving model to dir_01.h5
Epoch 2/500
346/346 [==============================] - 0s 1ms/step - loss: 0.1054 - acc: 0.9538 - val_loss: 0.0042 - val_acc: 0.9971

Epoch 00002: val_loss did not improve from 0.00415
Epoch 3/500
346/346 [==============================] - 0s 1ms/step - loss: 0.1261 - acc: 0.9509 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00003: val_loss improved from 0.00415 to 0.00407, saving model to dir_01.h5
Epoch 4/500
346/346 [==============================] - 0s 1ms/step - loss: 0.1091 - acc: 0.9624 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00004: val_loss did not improve from 0.00407
Epoch 5/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0723 - acc: 0.9740 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00005: val_loss 

346/346 [==============================] - 0s 1ms/step - loss: 0.0800 - acc: 0.9624 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00044: val_loss improved from 0.00406 to 0.00405, saving model to dir_01.h5
Epoch 45/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0894 - acc: 0.9509 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00045: val_loss did not improve from 0.00405
Epoch 46/500
346/346 [==============================] - 0s 994us/step - loss: 0.1009 - acc: 0.9566 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00046: val_loss did not improve from 0.00405
Epoch 47/500
346/346 [==============================] - 0s 989us/step - loss: 0.1015 - acc: 0.9595 - val_loss: 0.0042 - val_acc: 0.9971

Epoch 00047: val_loss did not improve from 0.00405
Epoch 48/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0852 - acc: 0.9653 - val_loss: 0.0045 - val_acc: 0.9971

Epoch 00048: val_loss did not improve from 0.00405
Epoch 49/500
346/346 [==========================

346/346 [==============================] - 0s 1ms/step - loss: 0.0767 - acc: 0.9711 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00088: val_loss did not improve from 0.00405
Epoch 89/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0691 - acc: 0.9711 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00089: val_loss did not improve from 0.00405
Epoch 90/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0562 - acc: 0.9798 - val_loss: 0.0042 - val_acc: 0.9971

Epoch 00090: val_loss did not improve from 0.00405
Epoch 91/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0701 - acc: 0.9711 - val_loss: 0.0042 - val_acc: 0.9971

Epoch 00091: val_loss did not improve from 0.00405
Epoch 92/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0806 - acc: 0.9653 - val_loss: 0.0043 - val_acc: 0.9971

Epoch 00092: val_loss did not improve from 0.00405
Epoch 93/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0804 

346/346 [==============================] - 0s 1ms/step - loss: 0.0691 - acc: 0.9740 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00132: val_loss did not improve from 0.00405
Epoch 133/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0597 - acc: 0.9827 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00133: val_loss did not improve from 0.00405
Epoch 134/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0832 - acc: 0.9653 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00134: val_loss did not improve from 0.00405
Epoch 135/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0453 - acc: 0.9769 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00135: val_loss did not improve from 0.00405
Epoch 136/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0832 - acc: 0.9624 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00136: val_loss did not improve from 0.00405
Epoch 137/500
346/346 [==============================] - 0s 1ms/step - loss: 0.

346/346 [==============================] - 0s 1ms/step - loss: 0.1397 - acc: 0.9509 - val_loss: 0.0064 - val_acc: 0.9971

Epoch 00176: val_loss did not improve from 0.00405
Epoch 177/500
346/346 [==============================] - 0s 1ms/step - loss: 0.1028 - acc: 0.9538 - val_loss: 0.0061 - val_acc: 0.9971

Epoch 00177: val_loss did not improve from 0.00405
Epoch 178/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0938 - acc: 0.9711 - val_loss: 0.0050 - val_acc: 0.9971

Epoch 00178: val_loss did not improve from 0.00405
Epoch 179/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0742 - acc: 0.9653 - val_loss: 0.0045 - val_acc: 0.9971

Epoch 00179: val_loss did not improve from 0.00405
Epoch 180/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0834 - acc: 0.9740 - val_loss: 0.0043 - val_acc: 0.9971

Epoch 00180: val_loss did not improve from 0.00405
Epoch 181/500
346/346 [==============================] - 0s 1ms/step - loss: 0.

...

346/346 [==============================] - 0s 1ms/step - loss: 0.0520 - acc: 0.9769 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00307: val_loss did not improve from 0.00403
Epoch 308/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0383 - acc: 0.9855 - val_loss: 0.0049 - val_acc: 0.9971

Epoch 00308: val_loss did not improve from 0.00403
Epoch 309/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0510 - acc: 0.9798 - val_loss: 0.0123 - val_acc: 0.9942

Epoch 00309: val_loss did not improve from 0.00403
Epoch 310/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0461 - acc: 0.9769 - val_loss: 0.0050 - val_acc: 0.9971

Epoch 00310: val_loss did not improve from 0.00403
Epoch 311/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0421 - acc: 0.9798 - val_loss: 0.0043 - val_acc: 0.9971

Epoch 00311: val_loss did not improve from 0.00403
Epoch 312/500
346/346 [==============================] - 0s 1ms/step - loss: 0.

346/346 [==============================] - 0s 1ms/step - loss: 0.0404 - acc: 0.9855 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00351: val_loss did not improve from 0.00403
Epoch 352/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0523 - acc: 0.9798 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00352: val_loss did not improve from 0.00403
Epoch 353/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0503 - acc: 0.9798 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00353: val_loss did not improve from 0.00403
Epoch 354/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0525 - acc: 0.9769 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00354: val_loss did not improve from 0.00403
Epoch 355/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0353 - acc: 0.9827 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00355: val_loss did not improve from 0.00403
Epoch 356/500
346/346 [==============================] - 0s 1ms/step - loss: 0.

346/346 [==============================] - 0s 1ms/step - loss: 0.0348 - acc: 0.9827 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00394: val_loss did not improve from 0.00402
Epoch 395/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0346 - acc: 0.9798 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00395: val_loss did not improve from 0.00402
Epoch 396/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0433 - acc: 0.9798 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00396: val_loss did not improve from 0.00402
Epoch 397/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0627 - acc: 0.9711 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00397: val_loss did not improve from 0.00402
Epoch 398/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0486 - acc: 0.9769 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00398: val_loss did not improve from 0.00402
Epoch 399/500
346/346 [==============================] - 0s 1ms/step - loss: 0.


Epoch 00437: val_loss did not improve from 0.00402
Epoch 438/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0539 - acc: 0.9827 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00438: val_loss did not improve from 0.00402
Epoch 439/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0704 - acc: 0.9711 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00439: val_loss did not improve from 0.00402
Epoch 440/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0434 - acc: 0.9798 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00440: val_loss did not improve from 0.00402
Epoch 441/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0328 - acc: 0.9855 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00441: val_loss did not improve from 0.00402
Epoch 442/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0514 - acc: 0.9769 - val_loss: 0.0041 - val_acc: 0.9971

Epoch 00442: val_loss did not improve from 0.00402
Epoch 443/500

346/346 [==============================] - 0s 1ms/step - loss: 0.0350 - acc: 0.9884 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00481: val_loss did not improve from 0.00401
Epoch 482/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0224 - acc: 0.9913 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00482: val_loss did not improve from 0.00401
Epoch 483/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0398 - acc: 0.9769 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00483: val_loss did not improve from 0.00401
Epoch 484/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0383 - acc: 0.9827 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00484: val_loss did not improve from 0.00401
Epoch 485/500
346/346 [==============================] - 0s 1ms/step - loss: 0.0337 - acc: 0.9913 - val_loss: 0.0040 - val_acc: 0.9971

Epoch 00485: val_loss did not improve from 0.00401
Epoch 486/500
346/346 [==============================] - 0s 1ms/step - loss: 0.

In [61]:
 model = load_model("dir_01.h5")

In [62]:
def predictions(text):
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    test_word = word_tokenize(clean)
    test_word = [w.lower() for w in test_word]
    test_ls = token.texts_to_sequences(test_word)
    #print(test_word)
    #Check for unknown words
    if [] in test_ls:
        test_ls = list(filter(None, test_ls))
    
    test_ls = np.array(test_ls).reshape(1, len(test_ls))

    #print("test_ls: ", test_ls)

    x = pad_sequences(test_ls, maxlen = MAX_SENT_LENGTH, padding = "post")
    
    # print ("x: ", x)
    
    pred = model.predict_proba(x)
  
    return pred


In [63]:
def get_final_output(pred, classes):
    #print (type (pred))
    #print (pred)
    
    predictions = pred[0]
 
    classes = np.array(classes)
    ids = np.argsort(-predictions)
    classes = classes[ids]
#    predictions = -np.sort(-predictions)

#    for i in range(pred.shape[1]):
#        print("%s has confidence = %s" % (classes[i], (predictions[i])))
    
    return classes[0]


In [64]:
text = "Good morning"
pred = predictions(text)
out = get_final_output(pred, intents_unique)
print (out)

greet


In [66]:
# with open('mowgli_test_new.csv', newline='') as csvfile:
#    testfilelist = list(csv.reader(csvfile))

with open('mowgli_train_new.csv', newline='') as csvfile:
    testfilelist = list(csv.reader(csvfile))

total   = 0
correct = 0

for s in testfilelist:
    right_label = s[1]
    message     = s[0]

    pred   = predictions(message)
    intent = get_final_output(pred, intents_unique)

    if intent == right_label:
        correct += 1
    else:
        print ("%s recognized as %s, correct is %s" % (message, intent, right_label))
    total += 1

print ("Results: %d from %d correct = %4.2f percent" % (correct, total, (correct/total)*100.0))


cool recognized as thanks, correct is confirm
Results: 345 from 346 correct = 99.71 percent
